In [20]:
from astroquery.sdss import SDSS
from astropy import units as u
from astropy import coordinates as coords
from scipy.ndimage.filters import uniform_filter1d
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
plt.rc('text', usetex=True)
plt.rcParams.update({'font.size': 22})
import numpy as np
pos = coords.SkyCoord('0h8m05.63s +14d50m23.3s', frame='icrs')
print(pos)
xid = SDSS.query_region(pos, radius=2*u.arcsec, spectro=True)
print(xid)
from astropy.cosmology import Planck15
clight = 0.306601393788/1e6
print(clight)
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

<SkyCoord (ICRS): (ra, dec) in deg
    (2.02345833, 14.83980556)>
       ra              dec               objid        ... run2d instrument
---------------- ---------------- ------------------- ... ----- ----------
2.02344596573482 14.8398237551311 1237652943176138868 ...    26       SDSS
3.06601393788e-07


In [21]:
def running_mean(y_in, x_in, N_out=101, sigma=1):
    '''
    from https://stackoverflow.com/questions/13728392/moving-average-or-running-mean/43200476#43200476
    Returns running mean as a Bell-curve weighted average at evenly spaced
    points. Does NOT wrap signal around, or pad with zeros.

    Arguments:
    y_in -- y values, the values to be smoothed and re-sampled
    x_in -- x values for array

    Keyword arguments:
    N_out -- NoOf elements in resampled array.
    sigma -- 'Width' of Bell-curve in units of param x .
    '''
    N_in = len(y_in)

    # Gaussian kernel
    x_out = np.linspace(np.min(x_in), np.max(x_in), N_out)
    x_in_mesh, x_out_mesh = np.meshgrid(x_in, x_out)
    gauss_kernel = np.exp(-np.square(x_in_mesh - x_out_mesh) / (2 * sigma**2))
    # Normalize kernel, such that the sum is one along axis 1
    normalization = np.tile(np.reshape(np.sum(gauss_kernel, axis=1), (N_out, 1)), (1, N_in))
    gauss_kernel_normalized = gauss_kernel / normalization
    # Perform running average as a linear operation
    y_out = gauss_kernel_normalized @ y_in

    return y_out, x_out

We are going to make a SQL query from the SDSS sky server to get galaxy spectra

In [35]:
query = "select top 2000 \
    objid, mjd, z, ra, dec, \
    psfmag_i, \
    extinction_i, \
    psfmag_r, \
    extinction_r \
    from \
    specPhoto \
    where \
    class = 'galaxy' \
    and psfmag_r-extinction_r <20 \
    and zWarning = 0"
res = SDSS.query_sql(query)
xid = SDSS.query_region(pos, radius=2*u.arcsec, spectro=True)

In [36]:
f = open('Summary_info.txt', 'w')
f.write(' ID \t ra \t dec \t extinction \t norm brightness \t SNR proxy \t Halpha pos \t z  \n')
f.close()
ind = 0
spec_count=0

while spec_count < 200:   #only want 20
    c =  coords.SkyCoord(ra=res['ra'][ind]*u.degree, dec=res['dec'][ind]*u.degree, frame='icrs')
    xid = SDSS.query_region(c, spectro=True) 
    try:
        # if there is a spectrum available at that sky location with bright Halpha
        sp = SDSS.get_spectra(matches=xid, mjd=res['mjd'][ind])
        halpha_ind = np.where(np.abs(10.**sp[0][1].data['loglam']-6563*(1+res['z'][ind]))<1)[0][0]
        
        if (sp[0][1].data['flux'][halpha_ind] > 4*np.mean(sp[0][1].data['flux'])):
            # the line is brighter than 4x the average flux
            print('line > signal', spec_count, 'spec_count')
            spec_count+=1
            print(sp[0][1].data['flux'][halpha_ind], np.mean(sp[0][1].data['flux']), 'fluxes')
            dist = Planck15.comoving_distance(res['z'][ind]).value #compute distance
            fluxx= 2.512**res['psfmag_r'][ind]/1e7
            snrprox= sp[0][1].data['flux'][halpha_ind]/np.mean(sp[0][1].data['flux'])
            print(res['z'][ind], dist, 'dist', fluxx, 'flux')
            print(res['objid'][ind], res['ra'][ind], res['dec'][ind],res['extinction_i'][ind],\
                  modflux, snrprox, 6563*(1+res['z'][ind]), res['z'][ind])
            ff = open('Summary_info.txt', 'a+')
            ff.write('%s \t %2.3f \t %2.3f \t %2.3f \t %2.3f \t %3.2f \t %2.3f \t %2.3f \n'\
                     %(str(res['objid'][ind]), res['ra'][ind], res['dec'][ind],\
                       res['extinction_i'][ind], modflux, snrprox, 6563*(1+res['z'][ind]),\
                       res['z'][ind]))
            ff.close()
            
            fig, ax = plt.subplots(figsize=(20.,15)) 
            running_mean_y, running_mean_x = running_mean(sp[0][1].data['flux'],10.**sp[0][1].data['loglam'], N_out=30, sigma=1) 
            fav_y = interp1d(running_mean_x, running_mean_y)
            
            av_y =fav_y(10.**sp[0][1].data['loglam'] )
            ax.plot(10.**sp[0][1].data['loglam'], sp[0][1].data['flux']/av_y, label='d=%2.3f Mpc, normalized apparent brightness = %2.3f' %(dist, fluxx), color='k')
            ax.plot(6563*(1+res['z'][ind])*np.ones(2), [0.1*np.min(sp[0][1].data['flux']/av_y), 1.5*np.max(sp[0][1].data['flux']/av_y)], '--', lw=3, color='red')
            plt.text(6563*(1+res['z'][ind])+450, 1.00*np.max(sp[0][1].data['flux']/av_y), 'Hydrogen line', fontsize=40)
            plt.annotate("", xy=(6563*(1+res['z'][ind])+400, np.max(sp[0][1].data['flux']/av_y)), xytext=(6563*(1+res['z'][ind]), np.max(sp[0][1].data['flux']/av_y)), 
                         arrowprops={'arrowstyle':"<-"})
            ax.set_ylabel('Flux [10$^{-27}$ ergs/cm$^2$/s/m]', fontsize=40)
            plt.title('SDSS Galaxy: ID%s'%str(res['objid'][ind]), fontsize=40)
            ax.axis([np.min(10.**sp[0][1].data['loglam']), np.max(10.**sp[0][1].data['loglam']),0.1*np.min(sp[0][1].data['flux']/av_y), 1.5*np.max(sp[0][1].data['flux']/av_y) ])
            ax.set_xlabel('Wavelength [$10^{-10}$m]', fontsize=40)
            ax.xaxis.set_minor_locator(AutoMinorLocator(20))
            for tick in ax.xaxis.get_major_ticks():
                tick.label.set_fontsize(35) 
            ax.tick_params(which='minor') 
            ax.legend(loc='upper right', fontsize=40)
            modflux = fluxx/4
            h0 = float(res['z'][ind])*clight/dist
            plt.savefig('%s_spec.png'%str(res['objid'][ind]))
            plt.close(fig)
    except:
        print('did not work for id %s'%res['objid'][ind])
        pass

    ind+=1

line > signal 0 spec_count
38.966 6.629201 fluxes
0.1804788 764.3763900261242 dist 5.494699131327007 flux
1237655500272894063 198.69453 -1.3697248 0.04763568 0.9591803415254196 5.877933 7747.4823644 0.1804788
line > signal 1 spec_count
100.99357 18.050573 fluxes
0.09524147 412.0415936738182 dist 1.847722233625859 flux
1237655495977599093 198.88077 -1.5249211 0.05003025 1.3736747828317517 5.5950336 7188.06976761 0.09524147
line > signal 2 spec_count
87.807594 7.6817603 fluxes
0.1033962 446.4281123329894 dist 6.2968423736933365 flux
1237655500272894086 198.73281 -1.3086334 0.04557665 0.46193055840646474 11.43066 7241.5892606 0.1033962
line > signal 3 spec_count
81.34574 15.728863 fluxes
0.08584301 372.23391910968706 dist 2.0978932788843068 flux
1237655495977795696 199.34741 -1.4468194 0.04650578 1.5742105934233341 5.1717496 7126.38767463 0.08584301
line > signal 4 spec_count
47.448364 11.574056 fluxes
0.09744804 421.36023469118413 dist 3.278392202580723 flux
1237655500273025213 199.0535 

line > signal 38 spec_count
41.921295 9.363043 fluxes
0.1088267 469.24803829382734 dist 3.600865568802523 flux
1237655495977861206 199.52587 -1.394623 0.05271676 0.4074684798569449 4.4773154 7277.2296321 0.1088267
line > signal 39 spec_count
83.98223 12.549647 fluxes
0.07967314 345.99851428938496 dist 2.61079068272592 flux
1237655495977795747 199.40138 -1.3947315 0.04921556 0.9002163922006308 6.6919994 7085.89481782 0.07967314
line > signal 40 spec_count
41.493843 9.198575 fluxes
0.1417369 606.1829602053242 dist 3.4887530878066673 flux
1237655495977861190 199.51282 -1.4742224 0.05494161 0.65269767068148 4.510899 7493.2192747 0.1417369
line > signal 41 spec_count
54.208885 10.80998 fluxes
0.08349306 362.2510813176063 dist 4.1388327446745 flux
1237655499736285474 199.33848 -1.6201318 0.04365482 0.8721882719516668 5.014707 7110.964952779999 0.08349306
line > signal 42 spec_count
60.52386 13.40655 fluxes
0.04779048 209.13919757638152 dist 2.926809728088964 flux
1237655499736416472 199.5624

line > signal 76 spec_count
100.4605 10.058584 fluxes
0.07872328 341.95232772062917 dist 4.1304544118607 flux
1237648704063602984 236.7585 -0.37704667 0.1584937 0.7649542219338468 9.987539 7079.660886639999 0.07872328
line > signal 77 spec_count
60.55735 14.156087 fluxes
0.07919856 343.97715220850387 dist 3.110196192610884 flux
1237648704063602698 236.768 -0.3601969 0.1584332 1.032613602965175 4.277831 7082.78014928 0.07919856
line > signal 78 spec_count
14.98949 3.5820708 fluxes
0.05530917 241.6076590829436 dist 11.45933949084238 flux
1237655693020430947 236.80667 -1.0304759 0.171906 0.777549048152721 4.1845875 6925.994082709999 0.05530917
line > signal 79 spec_count
120.32529 22.328255 fluxes
0.01498329 66.07960467508998 dist 2.4836534060373836 flux
1237655693020430340 236.80374 -0.99602017 0.1714172 2.864834872710595 5.388925 6661.3353322699995 0.01498329
line > signal 80 spec_count
155.29645 6.3557143 fluxes
0.08876021 384.6100884991004 dist 6.578816239152078 flux
12376487046003428

line > signal 109 spec_count
77.25532 16.829088 fluxes
0.06982577 303.95773606036795 dist 2.3148640459660172 flux
1237648705666154722 218.40333 0.84525214 0.06535412 0.8058914668272599 4.5905824 7021.26652851 0.06982577
line > signal 110 spec_count
40.015144 8.13642 fluxes
0.1157689 498.3280405043748 dist 5.59412671323505 flux
1237648705666154592 218.45979 0.87053392 0.06520306 0.5787160114915043 4.9180284 7322.7912907 0.1157689
line > signal 111 spec_count
73.79155 12.901732 fluxes
0.03527383 154.82411513147653 dist 3.5494298837984197 flux
1237648721784602889 218.87499 0.41367984 0.06763292 1.3985316783087625 5.719507 6794.5021462899995 0.03527383
line > signal 112 spec_count
462.47516 71.99315 fluxes
0.03455789 151.7074200264817 dist 0.5507809143188201 flux
1237648721784602712 218.85559 0.33433494 0.06578679 0.8873574709496049 6.4238772 6789.803432070001 0.03455789
line > signal 113 spec_count
135.20874 19.063763 fluxes
0.0266113 117.04172545616176 dist 2.514822027869757 flux
1237648

line > signal 147 spec_count
129.52367 17.455435 fluxes
0.03339099 146.6252681769375 dist 2.9557704279259345 flux
1237648721784930648 219.63677 0.41709185 0.08291662 1.0317674539565893 7.4202485 6782.14506737 0.03339099
line > signal 148 spec_count
67.40401 6.629621 fluxes
0.1302673 558.7250210744483 dist 6.2047245698264115 flux
1237648722321735967 219.46432 0.69643553 0.06941238 0.7389426069814836 10.167098 7417.9442899000005 0.1302673
line > signal 149 spec_count
78.19358 17.21995 fluxes
0.2106875 885.4607395355875 dist 2.1480673570267577 flux
1237648704592871475 219.43814 0.078787432 0.07982378 1.5511811424566029 4.5408716 7945.7420624999995 0.2106875
line > signal 150 spec_count
15.180861 2.759973 fluxes
0.2771648 1144.9186953023814 dist 10.070077916634936 flux
1237648722321867282 219.72502 0.83493253 0.07745852 0.5370168392566894 5.5003657 8382.0325824 0.2771648
line > signal 151 spec_count
58.145264 13.403219 fluxes
0.03434044 150.76058471658558 dist 3.779687908757463 flux
123764

line > signal 184 spec_count
75.47492 15.595241 fluxes
0.08290768 359.76249120374825 dist 2.7064550671080303 flux
1237650770440093827 201.87923 0.42437531 0.04490924 1.1132570263239792 4.8396125 7107.123103839999 0.08290768
line > signal 185 spec_count
40.107403 9.301521 fluxes
0.08469289 367.3495597149996 dist 3.450467956965098 flux
1237648722314002610 201.79899 0.68753995 0.0470055 0.6766137667770076 4.3119187 7118.83943707 0.08469289
line > signal 186 spec_count
56.9942 9.865574 fluxes
0.08152754 353.8922933659383 dist 4.757084263051858 flux
1237648722314002470 201.73155 0.69731024 0.04880855 0.8626169892412745 5.777079 7098.06524502 0.08152754
line > signal 187 spec_count
56.736694 9.512544 fluxes
0.0788918 342.67032467812606 dist 4.098204879643957 flux
1237648722314068122 201.88322 0.79538309 0.04767617 1.1892710657629646 5.9644084 7080.766883400001 0.0788918
line > signal 188 spec_count
39.76443 8.5537615 fluxes
0.08355405 362.51032231053784 dist 5.223947935959396 flux
1237651504